# Test: mass traced by the different molecules


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
import astropy.units as u
from astropy.constants import k_B, h, c
from scipy import stats
%matplotlib inline

## HC$_3$N
We first calculate the column density of HC$_3$N (10-9)

In [2]:
[hc3nred_ampl, hc3nred_vel, hc3nred_sigma, ehc3nred_ampl, ehc3nred_vel, ehc3nred_sigma], hc3nred_head = fits.getdata('../clustering/clusters_red_HC3N.fits', header=True)
[hc3nblue_ampl, hc3nblue_vel, hc3nblue_sigma, ehc3nblue_ampl, ehc3nblue_vel, ehc3nblue_sigma], hc3nblue_head = fits.getdata('../clustering/clusters_blue_HC3N.fits', header=True)

headerhc3nflat = fits.getheader('../data/NGC1333_HC3N_L24-merged_small_K_5.0_10.0_mom0.fits')

In [3]:
hc3nred_mom0 = hc3nred_ampl * hc3nred_sigma * np.sqrt(2*np.pi) * u.K * u.km/u.s
e_hc3nred_mom0 = hc3nred_mom0 * np.sqrt((ehc3nred_ampl/hc3nred_ampl)**2 + (ehc3nred_sigma/hc3nred_sigma)**2)
hc3nblue_mom0 = hc3nblue_ampl * hc3nblue_sigma * np.sqrt(2*np.pi)* u.K * u.km/u.s
e_hc3nblue_mom0 = hc3nblue_mom0 * np.sqrt((ehc3nblue_ampl/hc3nblue_ampl)**2 + (ehc3nblue_sigma/hc3nblue_sigma)**2)

newhead_red = hc3nred_head.copy()
newhead_red['NAXIS3'] = 2
newhead_blue = hc3nblue_head.copy()
newhead_blue['NAXIS3'] = 2

In [4]:
fits.writeto('HC3N_10_9_mom0_red.fits', [hc3nred_mom0.value, e_hc3nred_mom0.value], newhead_red, overwrite=True)
fits.writeto('HC3N_10_9_mom0_blue.fits', [hc3nblue_mom0.value, e_hc3nblue_mom0.value], newhead_blue, overwrite=True)

In [5]:
# here we calculate the N(HC3N) of the 10-9 level

nu_HC3N109 = 90.970 * u.GHz
J = 10
S = J**2 / (J * (2*J + 1))
Aul = 0.581e-4/u.s

hc3nred_Ncol = (hc3nred_mom0 * 8 * np.pi * k_B * nu_HC3N109**2 / (h * c**3 * Aul)).to(1/u.cm**2)
e_hc3nred_Ncol = (e_hc3nred_mom0 * 8 * np.pi * k_B * nu_HC3N109**2 / (h * c**3 * Aul)).to(1/u.cm**2)
hc3nblue_Ncol = (hc3nblue_mom0 * 8 * np.pi * k_B * nu_HC3N109**2 / (h * c**3 * Aul)).to(1/u.cm**2)
e_hc3nblue_Ncol = (e_hc3nblue_mom0 * 8 * np.pi * k_B * nu_HC3N109**2 / (h * c**3 * Aul)).to(1/u.cm**2)



In [6]:
fits.writeto('HC3N_10_9_ncol_red.fits', [hc3nred_Ncol.value, e_hc3nred_Ncol.value], newhead_red, overwrite=True)
fits.writeto('HC3N_10_9_ncol_blue.fits', [hc3nblue_Ncol.value, e_hc3nblue_Ncol.value], newhead_blue, overwrite=True)

In [7]:
# now we transform to total column density of the molecule
Tex = 12.4 * u.K
Eup = 24.01482 * u.K #CDMS
g_up = 2 * J +1
B0 = 4549.0579 * u.MHz

expfactor = (h*B0/(3*k_B*Tex)).decompose().value
Qrot = k_B * Tex/ (h*B0) * np.exp(expfactor)
pop_up_constant = (Qrot/g_up * np.exp(Eup.value/(Tex.value))).decompose().value


In [8]:
hc3nred_Ncol_total = pop_up_constant * hc3nred_Ncol
e_hc3nred_Ncol_total = pop_up_constant * e_hc3nred_Ncol

hc3nblue_Ncol_total = pop_up_constant * hc3nblue_Ncol
e_hc3nblue_Ncol_total = pop_up_constant * e_hc3nblue_Ncol

In [9]:
fits.writeto('HC3N_Ncol_red.fits', [hc3nred_Ncol_total.value, e_hc3nred_Ncol_total.value], newhead_red, overwrite=True)
fits.writeto('HC3N_Ncol_blue.fits', [hc3nblue_Ncol_total.value, e_hc3nblue_Ncol_total.value], newhead_blue, overwrite=True)

In [10]:
# now we check how much H2 gas is tracing
abundance_HC3N_TMC1 = 2.8e-9
#abundance_Per2 = ??

NH2_red = hc3nred_Ncol_total / abundance_HC3N_TMC1
e_NH2_red = e_hc3nred_Ncol_total / abundance_HC3N_TMC1
NH2_blue = hc3nblue_Ncol_total / abundance_HC3N_TMC1
e_NH2_blue = e_hc3nblue_Ncol_total / abundance_HC3N_TMC1

In [11]:
fits.writeto('HC3N_NH2_red.fits', [NH2_red.value, e_NH2_red.value], newhead_red, overwrite=True)
fits.writeto('HC3N_NH2_blue.fits', [NH2_blue.value, e_NH2_blue.value], newhead_blue, overwrite=True)

With the total column density, we can estimate the mass

In [12]:
from astropy.constants import u as amu
mu_mass = 2.8
distance = 293 #pc
A_pixel = (hc3nred_head['CDELT2'] * 3600 * distance * u.au)**2

M_red = np.nansum(NH2_red * mu_mass * amu * A_pixel).to(u.Msun)
M_blue = np.nansum(NH2_blue * mu_mass * amu * A_pixel).to(u.Msun)

In [13]:
print(M_red, M_blue)

0.8936087441969237 solMass 0.1096186560847292 solMass


# N$_2$H$^+$

For N$_2$H$^+$ the calculation is slightly different. We integrate the hyperfine component we used for analysis and then apply the statistic weight of the component to find the total flux

In [14]:
[n2hpred_ampl, n2hpred_vel, n2hpred_sigma, en2hpred_ampl, en2hpred_vel, en2hpred_sigma], n2hpred_head = fits.getdata('../clustering/clusters_red_N2Hp.fits', header=True)
[n2hpblue_ampl, n2hpblue_vel, n2hpblue_sigma, en2hpblue_ampl, en2hpblue_vel, en2hpblue_sigma], n2hpblue_head = fits.getdata('../clustering/clusters_blue_N2Hp.fits', header=True)


In [15]:
def Rayleigh_Jeans_J(T, nu):
    expo = (h * nu/(k_B * T)).decompose().value
    return (h * nu / (k_B)).to(u.K) / (np.exp(expo)-1)

# we wll use equation n. 96 from Mangum and Shirley 2015

In [16]:
n2hpred_mom0 = n2hpred_ampl * n2hpred_sigma * np.sqrt(2*np.pi) * u.K * u.km/u.s
e_n2hpred_mom0 = n2hpred_mom0 * np.sqrt((en2hpred_ampl/n2hpred_ampl)**2 + (en2hpred_sigma/n2hpred_sigma)**2)

n2hpblue_mom0 = n2hpblue_ampl * n2hpblue_sigma * np.sqrt(2*np.pi) * u.K * u.km/u.s
e_n2hpblue_mom0 = n2hpblue_mom0 * np.sqrt((en2hpblue_ampl/n2hpblue_ampl)**2 + (en2hpblue_sigma/n2hpblue_sigma)**2)


In [17]:
newhead_red_n2hp = n2hpred_head.copy()
newhead_red_n2hp['naxis3'] = 2
newhead_blue_n2hp = n2hpblue_head.copy()
newhead_blue_n2hp['naxis3'] = 2

In [18]:
fits.writeto('N2Hp_10_mom0_red.fits', [n2hpred_mom0.value, e_n2hpred_mom0.value], newhead_red_n2hp, overwrite=True)
fits.writeto('N2Hp_10_mom0_blue.fits', [n2hpblue_mom0.value, e_n2hpblue_mom0.value], newhead_blue_n2hp, overwrite=True)

In [19]:
R_i = 1/9.
E_up_N2Hp = 4.47165 * u.K
nu_N2Hp10_i = 93.176265 * u.GHz
T_ex_N2Hp = 7 *u.K # from our 1 component fit to all hypercomponents
Aul_N2Hp = 10 ** (-5.39891) /u.s # (-4.67019) /u.s # from CDMS, its -5.39891 in SLAIM????
gup = 3
B0_N2Hp = 46586.867 * u.MHz
lam_n2hp = (c / nu_N2Hp10_i).to(u.m)
J_Tex = Rayleigh_Jeans_J(T_ex_N2Hp, nu_N2Hp10_i) - Rayleigh_Jeans_J(2.73*u.K, nu_N2Hp10_i)
hnukTmin = -(h*nu_N2Hp10_i/(k_B*T_ex_N2Hp)).decompose().value
Qrot = (k_B * T_ex_N2Hp / (h * B0_N2Hp)).decompose().value + 1/3
EupkT = (E_up_N2Hp/T_ex_N2Hp).decompose().value

# n2hpred_Ncol10 =(n2hpred_mom0 * 8 * np.pi * k_B * nu_N2Hp10_i**2 / (h * c**3 * Aul_N2Hp)).to(1/u.cm**2)
# e_n2hpred_Ncol = n2hpred_Ncol * e_n2hpred_mom0/n2hpred_mom0
# n2hpblue_Ncol10 =(n2hpblue_mom0 * 8 * np.pi * k_B * nu_N2Hp10_i**2 / (h * c**3 * Aul_N2Hp)).to(1/u.cm**2)
# e_n2hpblue_Ncol = n2hpblue_Ncol * e_n2hpblue_mom0/n2hpblue_mom0

# Formula from Caselli et al 2002b
n2hpred_Ncol = (8 * np.pi * n2hpred_mom0 / (lam_n2hp**3 * Aul * gup * R_i) * 1/J_Tex * Qrot * np.exp(EupkT) / (1 - np.exp(hnukTmin))).to(u.cm**-2)
e_n2hpred_Ncol = n2hpred_Ncol * e_n2hpred_mom0/n2hpred_mom0
n2hpblue_Ncol = (8 * np.pi * n2hpblue_mom0 / (lam_n2hp**3 * Aul * gup * R_i) * 1/J_Tex * Qrot * np.exp(EupkT) / (1 - np.exp(hnukTmin))).to(u.cm**-2)
e_n2hpblue_Ncol = n2hpblue_Ncol * e_n2hpblue_mom0/n2hpblue_mom0
# Formula from Mangum and Shirley 2015, eq. 96 here because in comparison to 7 K, 2.73 K is non-negligible background
# hnukT = (h*nu_N2Hp10_i/(k_B*T_ex_N2Hp)).decompose().value
# 
# 

# n2hpred_Ncol = 6.25e15 / (nu_N2Hp10_i.value * R_i) * np.exp(EupkT) / (np.exp(hnukT) - 1) * n2hpred_mom0.value / J_Tex / (u.cm**2)
# e_n2hpred_Ncol = n2hpred_Ncol * e_n2hpred_mom0/n2hpred_mom0
# n2hpblue_Ncol = 6.25e15 / (nu_N2Hp10_i.value * R_i) * np.exp(EupkT) / (np.exp(hnukT) - 1) * n2hpblue_mom0.value / J_Tex / (u.cm**2)
# e_n2hpblue_Ncol = n2hpblue_Ncol * e_n2hpblue_mom0/n2hpblue_mom0

In [20]:
fits.writeto('N2Hp_Ncol_red.fits', [n2hpred_Ncol.value, e_n2hpred_Ncol.value], newhead_red_n2hp, overwrite=True)
fits.writeto('N2Hp_Ncol_blue.fits', [n2hpblue_Ncol.value, e_n2hpblue_Ncol.value], newhead_blue_n2hp, overwrite=True)

We use an abundance X of about $2.5\times 10^{-10}$ from Hacar et al (2017) in NGC1333

In [21]:
# now we check how much H2 gas is tracing
abundance_N2Hp_NGC = 2.5e-10

NH2n2hp_red = n2hpred_Ncol / abundance_N2Hp_NGC
e_NH2n2hp_red = e_n2hpred_Ncol / abundance_N2Hp_NGC
NH2n2hp_blue = n2hpblue_Ncol / abundance_N2Hp_NGC
e_NH2n2hp_blue = e_n2hpblue_Ncol / abundance_N2Hp_NGC

In [22]:
fits.writeto('N2Hp_NH2_red.fits', [NH2n2hp_red.value, e_NH2n2hp_red.value], newhead_red_n2hp, overwrite=True)
fits.writeto('N2Hp_NH2_blue.fits', [NH2n2hp_blue.value, e_NH2n2hp_blue.value], newhead_blue_n2hp, overwrite=True)

We now estimate the total mass within the filaments

In [23]:
A_pixel = (n2hpred_head['CDELT2'] * 3600 * distance * u.au)**2

M_n2hpred = np.nansum(NH2n2hp_red * mu_mass * amu * A_pixel).to(u.Msun)
M_n2hpblue = np.nansum(NH2n2hp_blue * mu_mass * amu * A_pixel).to(u.Msun)

In [27]:
print(M_n2hpred, M_n2hpblue)

105.02167026144424 solMass 20.16635294463893 solMass


In [25]:
(n2hpred_head['CDELT2'] * 3600* distance * u.au )**2

<Quantity 71310.47836642 AU2>